In [2]:
import numpy as np 
import pandas as pd
import os, re
import matplotlib.pyplot as plt
from tqdm import tqdm
from numba import njit
from time import perf_counter as pc

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# plt.style.use(["notebook", "science"])
plt.rcParams["figure.figsize"] = [15, 7]
plt.rcParams["figure.dpi"] = 150
plt.rcParams["lines.linewidth"] = 2

np.set_printoptions(linewidth=200)
np.set_printoptions(precision=3)

In [19]:
map_path = "raw txt-input 4889 samples 10 nodes/"

dir_list = os.listdir(map_path) # all the files
paths = [file for file in dir_list if file.endswith(".txt")] # filtering out files which is not txt
# a list with arrays 
dataset = [np.loadtxt(map_path + path) for path in tqdm(paths)]

100%|██████████| 4889/4889 [00:01<00:00, 2717.24it/s]


Using 5 different seeds {0,1,2,3,4} to generate different perturbations to the original LHGR dataset. The result is a perturbed dataset that is 5 times larger than the original.

In [20]:
def generate_new_LHGR(LHGR, noise_factor=0, seed=0):
    np.random.seed(seed)
    noise = noise_factor * np.random.rand(*LHGR.shape)
    fft_data = np.fft.fft(LHGR) + noise
    frequency_data = np.fft.ifft(fft_data)
    return frequency_data.real

def generate_new_LHGRs(dataset, noise_factor=0, seed=0):
    new_LHGRs = []
    for i, data in enumerate(dataset):
        LHGR_perturbed = generate_new_LHGR(data, noise_factor, seed)
        new_LHGRs.append(LHGR_perturbed - noise_factor/2)
    return new_LHGRs
    
new_dataset = [generate_new_LHGRs(dataset, 20, seed) for seed in range(5)]
result = np.asanyarray(new_dataset, dtype=object)

In [22]:
np.save("raw npy-input 24445 samples 10 nodes/perturbed_LHGR_dataset.npy", result)
np.save("raw npy-input 30930 samples 10 nodes/perturbed_LHGR_dataset.npy", result)

In [23]:
dataset1 = np.load("raw npy-input 30930 samples 10 nodes/perturbed_LHGR_dataset.npy", allow_pickle=True)
dataset2 = np.load("raw npy-input 24445 samples 10 nodes/perturbed_LHGR_dataset.npy", allow_pickle=True)
total_dataset = np.concatenate([dataset1, dataset2], axis=-1).flatten() 

In [18]:
total_dataset[34787].shape

(34, 10)